In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Dataset
from sklearn.datasets import load_digits

# PyTorch
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import training_logger

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))])

trainset = torchvision.datasets.MNIST(root='~/data',
                                        train=True,
                                        download=True,
                                        transform=transform)

testset = torchvision.datasets.MNIST(root='~/data',
                                        train=False,
                                        download=True,
                                        transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 171766390.32it/s]

Extracting /root/data/MNIST/raw/train-images-idx3-ubyte.gz to /root/data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 120055197.05it/s]


Extracting /root/data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 44891769.91it/s]

Extracting /root/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 5824068.72it/s]


Extracting /root/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/data/MNIST/raw



In [3]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=100,
                                          shuffle=True,
                                          num_workers=2)

In [4]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=100,
                                         shuffle=False,
                                         num_workers=2)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # 28x28x32 -> 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3)  # 26x26x64 -> 24x24x64
        self.pool = nn.MaxPool2d(2, 2)  # 24x24x64 -> 12x12x64
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12 * 12 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [6]:
torch.cuda.is_available()

True

In [7]:
device = torch.device("cuda:0")

In [8]:
model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
epochs = 10
logger = training_logger.Train("402b466c-b1fa-49b1-a76c-097ff42bf35e")
logger.create_train(train_id="test_train", num_epochs=epochs)
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[{epoch + 1}, {i+1}] loss: {running_loss / 100:.2}')
            running_loss = 0.0
        logger.send_metrics(epoch + 1, "loss", running_loss / 100)
logger.end_train()

print('Finished Training')

Train created


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


[1, 100] loss: 2.2
[1, 200] loss: 1.5
[1, 300] loss: 0.83
[1, 400] loss: 0.62
[1, 500] loss: 0.54
[1, 600] loss: 0.51
[2, 100] loss: 0.48
[2, 200] loss: 0.43
[2, 300] loss: 0.41
[2, 400] loss: 0.38
[2, 500] loss: 0.39
[2, 600] loss: 0.36
[3, 100] loss: 0.35
[3, 200] loss: 0.33
[3, 300] loss: 0.32
[3, 400] loss: 0.31
[3, 500] loss: 0.31
[3, 600] loss: 0.29
[4, 100] loss: 0.3
[4, 200] loss: 0.27
[4, 300] loss: 0.28
[4, 400] loss: 0.28
[4, 500] loss: 0.27
[4, 600] loss: 0.25
[5, 100] loss: 0.25
[5, 200] loss: 0.26
[5, 300] loss: 0.24
[5, 400] loss: 0.24
[5, 500] loss: 0.24
[5, 600] loss: 0.24
[6, 100] loss: 0.23
[6, 200] loss: 0.22
[6, 300] loss: 0.22
[6, 400] loss: 0.21
[6, 500] loss: 0.21
[6, 600] loss: 0.21
[7, 100] loss: 0.21
[7, 200] loss: 0.21
[7, 300] loss: 0.19
[7, 400] loss: 0.19
[7, 500] loss: 0.2
[7, 600] loss: 0.19
[8, 100] loss: 0.2
[8, 200] loss: 0.18
[8, 300] loss: 0.18
[8, 400] loss: 0.18
[8, 500] loss: 0.18
[8, 600] loss: 0.17
[9, 100] loss: 0.18
[9, 200] loss: 0.17
[9, 3